In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import transformers
import torch
from datasets import load_dataset
import transformers
from transformers import AutoFeatureExtractor, Wav2Vec2BertModel
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')


In [55]:
dataset = load_dataset("empathetic_dialogues")


In [56]:
train = pd.DataFrame(dataset['train'])
validation = pd.DataFrame(dataset['validation'])
test = pd.DataFrame(dataset['test'])

In [57]:
train

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,
...,...,...,...,...,...,...,...,...
76668,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5,
76669,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5,
76670,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5,
76671,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5,


In [58]:
# if you want to train faster with less data

# train = train[train['context'].isin(['sentimental','surprised'])].sample(100).reset_index(drop = True)
# validation = validation[validation['context'].isin(['sentimental','surprised'])].sample(20).reset_index(drop = True)
# test = test[test['context'].isin(['sentimental','surprised'])].sample(20).reset_index(drop = True)

In [59]:
train.shape,validation.shape,test.shape

((100, 8), (20, 8), (20, 8))

In [60]:
train['context'].unique(),validation['context'].unique(), test['context'].unique()

(array(['surprised', 'sentimental'], dtype=object),
 array(['sentimental', 'surprised'], dtype=object),
 array(['sentimental', 'surprised'], dtype=object))

In [61]:
train["text"] = train['prompt']+" [SEP] "+ train['utterance']
validation["text"] = validation['prompt']+" [SEP] "+ validation['utterance']
test["text"] = test['prompt']+" [SEP] "+ test['utterance']

In [62]:
# Create a label (category) encoder object
le = LabelEncoder()

le.fit(train["context"])

train_labels = le.transform(train["context"])
validation_labels = le.transform(validation["context"])
test_labels = le.transform(test["context"])

In [63]:
train["context"].unique()

array(['surprised', 'sentimental'], dtype=object)

In [64]:
class ConvAI2Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.long()  # Convert labels to Long

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [65]:
# Tokenize train data
train_encodings = tokenizer(train["text"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
train_dataset = ConvAI2Dataset(train_encodings, torch.tensor(train_labels).long())

# Tokenize validation data
validation_encodings = tokenizer(validation["text"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
validation_dataset = ConvAI2Dataset(validation_encodings, torch.tensor(validation_labels).long())

# Tokenize test data
test_encodings = tokenizer(test["text"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_dataset = ConvAI2Dataset(test_encodings, torch.tensor(test_labels).long())


In [66]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
training_args = TrainingArguments(
    output_dir='./BERT_empathetic_dialogues',
    num_train_epochs=2,#3,
    per_device_train_batch_size=2,#16,
    warmup_steps=10, #500,
    weight_decay= 0.1, #0.01,
    logging_dir='./logs',
    push_to_hub=True,
    evaluation_strategy="steps" # or "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset  # Add the validation dataset here
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=100, training_loss=0.5754035568237305, metrics={'train_runtime': 371.5861, 'train_samples_per_second': 0.538, 'train_steps_per_second': 0.269, 'total_flos': 11613886428000.0, 'train_loss': 0.5754035568237305, 'epoch': 2.0})

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [69]:
# Save the trained model
# trainer.save_model("./custom_mbert")

In [70]:
from sklearn.metrics import f1_score,confusion_matrix

# Make predictions on the test set
model.eval()
predictions = []
for item in test_dataset:
    # Move input data to the same device as the model
    input_ids = item['input_ids'].to(device).unsqueeze(0)
    attention_mask = item['attention_mask'].to(device).unsqueeze(0)

    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)
    logits = outputs.logits
    predictions.append(logits.argmax(-1).item())

# Assuming test_labels is a numpy array or a list. If it's a tensor, it needs to be on CPU and converted to numpy.
if isinstance(test_labels, torch.Tensor):
    test_labels = test_labels.cpu().numpy()

# Calculate F1 score
f1 = f1_score(test_labels, predictions, average='weighted')


In [71]:
f1

0.9

In [72]:
confusion_matrix(test_labels, predictions)

array([[ 6,  1],
       [ 1, 12]])

In [ ]:
# # Example prediction in encoded format
# predicted_labels_encoded = [0, 2, 1]

# # Convert back to original string format
# predicted_labels = le.inverse_transform(predicted_labels_encoded)

# print("Predicted labels in original format:", predicted_labels)